In [2]:
import os

import numpy as np
import scipy.io

In [3]:
# Read file and define the header
filename = '../data/recordings/s2_5_sf10.raw'
file_size = os.path.getsize(filename)

### Basic Exploration

In [32]:
# Let's define the header
header_t = np.dtype([('start_time', np.uint64),('center_freq', np.double),('gain', np.double),('sample_freq', np.double)])
header = np.fromfile(filename, dtype=header_t, count=1, sep='', offset=0)
header_size = header.itemsize

# Saving with our beloved variable names
start_time = header[0][0]
fc = header[0][1]
gain = header[0][2]
bw = header[0][3]

In [33]:
# Get the approximate number of chunks by reading the first
bytes64 = 8 # number of bytes in a 64 bit type
chunk_length = np.fromfile(filename, dtype=np.uint64, count=1, offset=header_size + 16)
chunk_size = 3 * bytes64 + 2 * (bytes64//2) * chunk_length

nchunks = (file_size - header_size) // chunk_size

In [19]:
# Reserve arrays
seconds = np.empty(nchunks, dtype=int)
nanoseconds = np.empty(nchunks, dtype=np.float64)
samples = np.empty(((chunk_length * nchunks)[0],2), dtype=np.float64)

In [28]:
offset = header_size

i = 0
while offset < file_size:
    # subheaders of the chunk
    sec   = np.fromfile(filename, dtype=np.uint64 , count=1, offset=offset)
    nsec  = np.fromfile(filename, dtype=np.float64, count=1, offset=offset+bytes64)
    nsamp = np.fromfile(filename, dtype=np.uint64 , count=1, offset=offset+2*bytes64)

    # flat array with the i/q samples of the chunk
    samps_flat = np.fromfile(filename, dtype=np.float32, count=int(nsamp*2), offset=offset+3*bytes64)
    samps_i = samps_flat[0::2]
    samps_q = samps_flat[1::2]

    # store our precious data on our result vector
    seconds[i] = sec
    nanoseconds[i] = nsec
    samples[int(i*nsamp):int((i+1)*nsamp),0] = samps_i
    samples[int(i*nsamp):int((i+1)*nsamp),1] = samps_q

    i += 1
    offset += int(3 * bytes64 + 2 * (bytes64//2) * nsamp)


In [34]:
scipy.io.savemat('test.mat', {'bw': bw, 'fc': fc, 'gain': gain, 'start_time': start_time, 'chunk_length': chunk_length, 'seconds': seconds, 'nanoseconds': nanoseconds, 'samples': samples})

### Batch Processing

In [36]:
scan_dir = '../data/receivers'

In [38]:
for file in os.scandir(scan_dir):
    file_name = file.name.split('.')[0]
    full_path = os.path.join(scan_dir,file.name)

    # Let's define the header
    header_t = np.dtype([('start_time', np.uint64),('center_freq', np.double),('gain', np.double),('sample_freq', np.double)])
    header = np.fromfile(full_path, dtype=header_t, count=1, sep='', offset=0)
    header_size = header.itemsize

    # Saving with our beloved variable names
    start_time = header[0][0]
    fc = header[0][1]
    gain = header[0][2]
    bw = header[0][3]

    # Get the approximate number of chunks by reading the first
    bytes64 = 8 # number of bytes in a 64 bit type
    chunk_length = np.fromfile(full_path, dtype=np.uint64, count=1, offset=header_size + 16)
    chunk_size = 3 * bytes64 + 2 * (bytes64//2) * chunk_length

    nchunks = (file_size - header_size) // chunk_size

    # Reserve arrays
    seconds = np.empty(nchunks, dtype=int)
    nanoseconds = np.empty(nchunks, dtype=np.float64)
    samples = np.empty(((chunk_length * nchunks)[0],2), dtype=np.float64)

    # main loop
    offset = header_size

    i = 0
    while offset < file_size:
        # subheaders of the chunk
        sec   = np.fromfile(full_path, dtype=np.uint64 , count=1, offset=offset)
        nsec  = np.fromfile(full_path, dtype=np.float64, count=1, offset=offset+bytes64)
        nsamp = np.fromfile(full_path, dtype=np.uint64 , count=1, offset=offset+2*bytes64)

        # flat array with the i/q samples of the chunk
        samps_flat = np.fromfile(full_path, dtype=np.float32, count=int(nsamp*2), offset=offset+3*bytes64)
        samps_i = samps_flat[0::2]
        samps_q = samps_flat[1::2]

        # store our precious data on our result vector
        seconds[i] = sec
        nanoseconds[i] = nsec
        samples[int(i*nsamp):int((i+1)*nsamp),0] = samps_i
        samples[int(i*nsamp):int((i+1)*nsamp),1] = samps_q

        i += 1
        offset += int(3 * bytes64 + 2 * (bytes64//2) * nsamp)
    
    # save to the cursed .mat format
    scipy.io.savemat(f'../data/matfiles/{file_name}.mat', {'bw': bw, 'fc': fc, 'gain': gain, 'start_time': start_time, 'chunk_length': chunk_length, 'seconds': seconds, 'nanoseconds': nanoseconds, 'samples': samples})
    print(f'Saved file: ../data/matfiles/{file_name}.mat')

FileNotFoundError: [Errno 2] No such file or directory: 's2_5_sf10.raw'